# Hyperparam Tuning

In [1]:
from keras.callbacks import EarlyStopping
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import to_categorical
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

%matplotlib inline

In [2]:
audio_data = np.load("../data/audio_data.npy")
labels = np.load("../data/wav_labels.npy")
NUM_SAMPLES = 1440
LOG_DIR = f"{int(time.time())}"

# labels: modality-vocal channel-emotion-emotional intensity-statement-repetition-actor
# emotions: 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
# odd number actors = male, even = female

# 1440 files: 24 speakers, 60 recordings per speaker
audio_data = audio_data.reshape(NUM_SAMPLES, 9480)

features = []

for i in range(1440):
    if (labels[i][6]%2 == 0):
        label = "Female"
    else:
        label = "Male"
    features.append([audio_data[i], label, int(labels[i][2])-1])
    
feature_df = pd.DataFrame(features, columns = ["mfcc", "gender", "emotion"])

feature_df.head()

,mfcc,gender,emotion
0,"[-710.0553588867188, -709.9026489257812, -711....",Female,0
1,"[-547.765625, -548.0353393554688, -548.6129760...",Female,2
2,"[-616.4595336914062, -615.7564697265625, -615....",Female,7
3,"[-739.8626098632812, -738.7739868164062, -735....",Female,1
4,"[-698.0630493164062, -697.3838500976562, -696....",Female,4


In [3]:
X = np.array(feature_df.mfcc.tolist())
y = np.array(feature_df.emotion.tolist())

scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(X)
X_scaled = scaler.transform(X)

#20-80 train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.10, random_state=0)

# Reshape for CNN input
X_train = np.array([x.reshape( (20, 474, 1) ) for x in X_train])
X_test = np.array([x.reshape( (20, 474, 1) ) for x in X_test])

# One-Hot encoding for classes
y_train = np.array(to_categorical(y_train, 8))
y_test = np.array(to_categorical(y_test, 8))

In [4]:
def build_model(hp):
    model = Sequential()

    MAX_SIZE = 96

    kernel_size = hp.Int('kernel_size', min_value=64, max_value=MAX_SIZE, step=4)
    stride_def = hp.Int('stride_def', min_value=2, max_value=8, step=1)
    model.add(Conv2D(kernel_size, (stride_def,stride_def), input_shape=X_test.shape[1:]))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2,2)))

    conv_1_units = hp.Int('conv_1_units', min_value=8, max_value=MAX_SIZE, step=8)
    model.add(Conv2D(conv_1_units, (3,3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(rate=hp.Int(f"dropout_val_1", 0, 2, 1) * 0.1))
    
    model.add(Conv2D(hp.Int('conv_2_units', min_value=8, max_value=conv_1_units, step=8), (3,3)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(rate=hp.Int(f"dropout_val_2", 0, 3, 1) * 0.1))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(LeakyReLU(alpha=0.1))

    model.add(Dropout(rate=hp.Int(f"dropout_end", 0, 5, 1) * 0.1))
    model.add(Dense(8))
    model.add(Activation('softmax'))

    opt = Adam(learning_rate=0.0005)

    model.compile(
        optimizer=opt,
        loss="categorical_crossentropy",
        metrics=['accuracy'])

    return model

In [5]:
tuner = RandomSearch(
    build_model,
    objective = "val_accuracy",
    max_trials = 20,
    executions_per_trial = 1,
    directory = LOG_DIR
)

tuner.search(
    x=X_train,
    y=y_train,
    epochs=150,
    batch_size=75,
    callbacks=[EarlyStopping(monitor='loss', patience=3)],
    validation_data=(X_test, y_test)
)

Trial 20 Complete [00h 00m 28s]
val_accuracy: 0.6527777910232544

Best val_accuracy So Far: 0.7361111044883728
Total elapsed time: 00h 09m 43s
INFO:tensorflow:Oracle triggered exit


In [8]:
with open(f"tuner_{int(time.time())}.pkl", "wb") as f:
    pickle.dump(tuner, f)

print(tuner.get_best_hyperparameters()[0].values)

{'kernel_size': 112, 'stride_def': 4, 'conv_1_units': 56, 'conv_2_units': 24, 'dropout_val_2': 2, 'conv_3_units': 128, 'dropout_val_3': 0}
